## ANN Regression Code

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [21]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [22]:
#Here we consider a continuous values feature (Est. salary) as an ouput because this is Regression ANN Model

#Delete unnecessary columns/ features
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
#Convert categorical values to numerical values

#Gender binary values case
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

#Geography multi-values case
onehot_encoder_geo = OneHotEncoder()
encoded_geo = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
encoded_geo_df = pd.DataFrame(encoded_geo,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
#Merge encoded Geography values to DF and drop original Geongraphy categ values
data = pd.concat([data.drop('Geography',axis=1),encoded_geo_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [24]:
#Save encoders models aas picke files

with open('label_encd_gen_Reg.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encd_geo_Reg.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [25]:
#Features I/P and O/P division
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [26]:
#Test and train split data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=40)

In [29]:
#Scale input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.23032791, -1.09003844,  3.6463151 , ..., -1.00601811,
         1.73089688, -0.57234647],
       [ 0.54015481,  0.91739884, -0.17964058, ...,  0.99401789,
        -0.57773517, -0.57234647],
       [ 0.46786187,  0.91739884, -0.65788503, ..., -1.00601811,
         1.73089688, -0.57234647],
       ...,
       [ 0.70539583, -1.09003844, -0.56223614, ..., -1.00601811,
        -0.57773517,  1.74719344],
       [-0.15179195, -1.09003844, -0.27528947, ..., -1.00601811,
        -0.57773517,  1.74719344],
       [ 0.45753431, -1.09003844, -0.75353393, ..., -1.00601811,
        -0.57773517,  1.74719344]], shape=(8000, 12))

In [30]:
#Save the scaler model as pickle file
with open ('scaler_Reg.pkl','wb') as file:
    pickle.dump(scaler,file)

##Build ANN Regression Model

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [32]:
#Set model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 connected with input layer
    Dense(32,activation='relu'), #HL2 automatically connected to HL1 as it is 'Sequential' model
    Dense(1) #NO ACTIVATION fn mean default 'LINEAR ACTIVATION fn'; this is O/P layer connected to HL2
])

#Compile model with loss fn and optimizer
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

c:\Users\sande\OneDrive\Desktop\DS ML NLP Krish Naik\Exercises\Practise NLP\NLP DL\ANN\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
#Set up TensorBoard logs
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Set up EarlyStopping 
earlystop_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [35]:
#Train the model using data

history = model.fit(X_train,y_train,epochs=100, validation_data=(X_test,y_test),
                    callbacks=[earlystop_callback,tensorboard_callback]
                    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - loss: 100204.8125 - mae: 100204.8125 - val_loss: 99300.0156 - val_mae: 99300.0156
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 99582.0312 - mae: 99582.0312 - val_loss: 98011.7344 - val_mae: 98011.7344
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 97339.8516 - mae: 97339.8516 - val_loss: 94653.8672 - val_mae: 94653.8672
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 92850.3438 - mae: 92850.3438 - val_loss: 88989.9688 - val_mae: 88989.9688
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 86154.3828 - mae: 86154.3828 - val_loss: 81390.1797 - val_mae: 81390.1797
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 77948.0625 - mae: 77948.0625 - val_loss: 73001.5156 - val_mae: 73001.5156
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 69420.3594 - mae: 69420.3594 - val_loss: 65321.2539 - val_mae: 65321.2539
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms

In [36]:
#Save model as .h5 file

model.save('model_reg.h5')

In [37]:
#Load TensorBoard extension to monitor logs

%load_ext tensorboard


In [40]:
%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6006 (pid 9764), started 0:00:15 ago. (Use '!kill 9764' to kill it.)

In [42]:
#Evaluate model on test data

test_loss, test_mae = model.evaluate(X_test,y_test)
print(f"TEST MAE: {test_mae}")
print(f"TEST LOSS: {test_loss}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50760.1055 - mae: 50760.1055
TEST MAE: 50760.10546875
TEST LOSS: 50760.10546875
